# importing libraries

In [2]:
import time
import redis
from redis import ConnectionPool
from redis.exceptions import ConnectionError
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import uuid
import multiprocessing
from dotenv import load_dotenv
import os

# Redis connection pool setup

In [3]:
# Load environment variables from .env file
load_dotenv()

# Get environment variables
REDIS_HOST = os.getenv('REDIS_HOST')
REDIS_PORT = int(os.getenv('REDIS_PORT'))
REDIS_USERNAME = os.getenv('REDIS_USERNAME')
REDIS_PASSWORD = os.getenv('REDIS_PASSWORD')

# Initialize the Redis connection pool
pool = redis.ConnectionPool(
    host=REDIS_HOST,
    port=REDIS_PORT,
    username=REDIS_USERNAME,
    password=REDIS_PASSWORD,
    max_connections=100,
    decode_responses=True
)

r = redis.Redis(connection_pool=pool)

# Prefixes for keys

In [3]:
ISSUE_PREFIX = "issue:"
VOTE_PREFIX = "vote:"

# Utility function for safe Redis operations with retries

In [4]:
def safe_redis_operation(operation, *args, **kwargs):
    retries = 3
    for attempt in range(retries):
        try:
            return operation(*args, **kwargs)
        except ConnectionError as e:
            if attempt < retries - 1:
                time.sleep(1)
                continue
            else:
                raise e

# Function to create an urban issue

In [5]:
def create_urban_issue(title: str, description: str, expires_in: int = 9000000) -> str:
    issue_id = str(uuid.uuid4())
    issue_key = ISSUE_PREFIX + issue_id
    safe_redis_operation(
        r.hset,
        issue_key,
        mapping={
            "title": title,
            "description": description,
            "expires_at": time.time() + expires_in
        }
    )
    safe_redis_operation(r.hset, issue_key + ":votes", "total_votes", 0)
    return issue_id

# Function to create a vote

In [6]:
def create_vote(issue_id: str, user_id: str) -> bool:
    issue_key = ISSUE_PREFIX + issue_id
    vote_key = VOTE_PREFIX + issue_id + ":" + user_id
    expires_at = safe_redis_operation(r.hget, issue_key, "expires_at")
    if not expires_at or time.time() > float(expires_at):
        return False
    if safe_redis_operation(r.exists, vote_key):
        return False  # User already voted
    safe_redis_operation(r.hincrby, issue_key + ":votes", "total_votes", 1)
    safe_redis_operation(r.set, vote_key, "VOTED")
    return True

# Function to update (toggle) a vote

In [7]:
def update_vote(issue_id: str, user_id: str) -> bool:
    issue_key = ISSUE_PREFIX + issue_id
    vote_key = VOTE_PREFIX + issue_id + ":" + user_id
    expires_at = safe_redis_operation(r.hget, issue_key, "expires_at")
    if not expires_at or time.time() > float(expires_at):
        return False
    if safe_redis_operation(r.exists, vote_key):
        safe_redis_operation(r.hincrby, issue_key + ":votes", "total_votes", -1)
        safe_redis_operation(r.delete, vote_key)
        return False  # Vote removed
    else:
        safe_redis_operation(r.hincrby, issue_key + ":votes", "total_votes", 1)
        safe_redis_operation(r.set, vote_key, "VOTED")
        return True  # Vote added

# Performance test for creating votes

In [8]:
def performance_test_create_votes(issue_id: str, num_votes: int, max_workers: int):
    start_time = time.time()
    def vote_task(i):
        user_id = f"user_{i}"
        create_vote(issue_id, user_id)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(vote_task, range(num_votes)), total=num_votes, desc="Creating votes"))
    elapsed = time.time() - start_time
    rps = num_votes / elapsed
    print(f"Created {num_votes} votes in {elapsed:.2f} seconds (~{rps:.2f} RPS)")

# Performance test for updating votes

In [9]:
def performance_test_update_votes(issue_id: str, num_users: int, max_workers: int):
    start_time = time.time()
    def update_task(i):
        user_id = f"user_{i}"
        update_vote(issue_id, user_id)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(update_task, range(num_users)), total=num_users, desc="Updating votes"))
    elapsed = time.time() - start_time
    rps = num_users / elapsed
    print(f"Updated {num_users} votes in {elapsed:.2f} seconds (~{rps:.2f} RPS)")

# Function to get vote results

In [10]:
def get_issue_results(issue_id: str) -> int:
    issue_key = ISSUE_PREFIX + issue_id + ":votes"
    return int(safe_redis_operation(r.hget, issue_key, "total_votes") or 0)

# Main execution

In [11]:
CPU_CORES = multiprocessing.cpu_count()
print(CPU_CORES)
issue_id = create_urban_issue("Traffic Congestion", "Heavy traffic near downtown", expires_in=9000000)
print(f"Urban Issue created with ID: {issue_id}")

# Performance test for creating votes
print("\nRunning performance test for creating votes...")
performance_test_create_votes(issue_id, num_votes=10000, max_workers=CPU_CORES)

votes = get_issue_results(issue_id)
print(f"\nTotal votes for Urban Issue {issue_id}: {votes}")
# Performance test for updating votes
print("\nRunning performance test for updating votes...")
performance_test_update_votes(issue_id, num_users=10000, max_workers=CPU_CORES)

# Display final vote count
votes = get_issue_results(issue_id)
print(f"\nTotal votes for Urban Issue {issue_id}: {votes}")

12
Urban Issue created with ID: a8d4d057-b318-42f9-9512-1dfc581bb991

Running performance test for creating votes...


Creating votes: 100%|███| 10000/10000 [05:07<00:00, 32.48it/s]


Created 10000 votes in 308.00 seconds (~32.47 RPS)

Total votes for Urban Issue a8d4d057-b318-42f9-9512-1dfc581bb991: 10000

Running performance test for updating votes...


Updating votes: 100%|███| 10000/10000 [05:08<00:00, 32.45it/s]

Updated 10000 votes in 308.22 seconds (~32.44 RPS)

Total votes for Urban Issue a8d4d057-b318-42f9-9512-1dfc581bb991: 0
